In [ ]:
"""
TRANSMORPHER: TRANSFORMER-BASED MORPHOLOGY PARSER
==================================================

This notebook implements a Transformer encoder-decoder model for morphological segmentation
of Quechua words. Unlike segmenter.ipynb which uses BiLSTM for boundary prediction, this
notebook uses a sequence-to-sequence Transformer architecture that directly generates
segmented words with '+' separators.

Key Features:
- Transformer encoder-decoder architecture (sequence-to-sequence)
- Character-level input/output tokenization
- Direct generation of segmented words (e.g., "pikunas" -> "pi+kuna+s")
- Data analysis section: correlation analysis, outlier detection, regression models
- Model checkpointing to avoid redundant training
- Comprehensive evaluation metrics (precision, recall, F1, exact match, split-count accuracy)

Key Differences from segmenter.ipynb:
- Uses Transformer architecture instead of BiLSTM
- Generates segmented strings directly (with '+' separators) instead of predicting boundaries
- Sequence-to-sequence approach rather than sequence labeling
- Includes exploratory data analysis (correlation, regression, outlier detection)

All data is read from the 'data' folder and models are saved to the 'models_transmorpher' folder.
"""

import os
import ast
import json
import hashlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from matplotlib import cm
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pickle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# =========================
# DATA FOLDER CONFIGURATION
# =========================
# All data files should be read from and saved to the data folder
DATA_FOLDER = "data"

# Model folder named after this notebook
MODEL_NAME = "transmorpher"
MODELS_FOLDER = f"models_{MODEL_NAME}"

# Create folders if they don't exist
os.makedirs(DATA_FOLDER, exist_ok=True)
os.makedirs(MODELS_FOLDER, exist_ok=True)

# =========================
# LOAD GOLD STANDARD DATA
# =========================
# The gold standard dataset contains high-quality morphological segmentations
# This is the base training data for the Transformer model
print("Loading gold standard data...")
gold_df = pd.read_parquet(os.path.join(DATA_FOLDER, "Sue_kalt.parquet"))
gold_df['Word'] = gold_df['word']
gold_df['morph'] = gold_df['morph'].str.replace('-', ' ')  # Normalize separators
gold_df['Morph_split_str'] = gold_df['morph']  # String version
gold_df['Morph_split'] = gold_df['morph'].str.split(' ')  # List version
gold_df = gold_df[['Word', 'Morph_split', 'Morph_split_str']]
gold_df.drop_duplicates(subset='Word', keep='first', inplace=True)
gold_df.dropna(subset=['Word'], inplace=True)
print(f"Loaded {len(gold_df):,} gold standard examples")

In [ ]:
gold_df.head()

In [ ]:
gold_df.shape

In [ ]:
gold_df['num_morphemes'] = gold_df['Morph_split'].apply(len)
gold_df['word_len'] = gold_df['Word'].apply(len)

In [ ]:
gold_df.head()

In [ ]:
gold_df.tail()

In [ ]:
heatmap_data = gold_df.groupby(['word_len', 'num_morphemes']).size().unstack(fill_value=0)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Heatmap of Word Length vs. Morpheme Count')
plt.xlabel('Number of Morphemes')
plt.ylabel('Word Length (Characters)')
plt.tight_layout()
plt.show()

In [ ]:
# Get the two series
x = gold_df['word_len']
y = gold_df['num_morphemes']

# Pearson (linear)
pearson_corr, pearson_p = pearsonr(x, y)

# Spearman (rank-based)
spearman_corr, spearman_p = spearmanr(x, y)

print(f"Pearson correlation: {pearson_corr:.3f} (p={pearson_p:.3e})")
print(f"Spearman correlation: {spearman_corr:.3f} (p={spearman_p:.3e})")

In [ ]:
gold_df1 = gold_df.copy()
print("Original Size: ", gold_df1.shape)

X = gold_df1[['word_len']]
y = gold_df1['num_morphemes']

model = LinearRegression()
model.fit(X, y)
gold_df1['predicted'] = model.predict(X)
gold_df1['residual'] = gold_df1['num_morphemes'] - gold_df1['predicted']

std_residual = gold_df1['residual'].std()
filtered_df = gold_df1[np.abs(gold_df1['residual']) <= std_residual]
print("Cleaned Size: ", filtered_df.shape)

X_filtered = filtered_df[['word_len']]
y_filtered = filtered_df['num_morphemes']

model_filtered = LinearRegression()
model_filtered.fit(X_filtered, y_filtered)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=filtered_df, x='word_len', y='num_morphemes', alpha=0.5, label='Filtered Data')
plt.plot(
    X_filtered, 
    model_filtered.predict(X_filtered), 
    color='red', 
    linewidth=2, 
    label='Regression Line'
)
plt.title('Optimized Linear Regression: Word Length vs Morpheme Count')
plt.xlabel('Word Length (Characters)')
plt.ylabel('Number of Morphemes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
slope = model.coef_[0]
intercept = model.intercept_
print(f"Pre_Refined Regression equation: num_morphemes ≈ {slope:.2f} × word_len + {intercept:.2f}")

slope = model_filtered.coef_[0]
intercept = model_filtered.intercept_
print(f"Refined Regression equation: num_morphemes ≈ {slope:.2f} × word_len + {intercept:.2f}")

In [ ]:
r2_full = r2_score(y, gold_df1['predicted'])
print(f"R2 (Original): {r2_full:.3f}")

y_pred_filtered = model_filtered.predict(X_filtered)
r2_filtered = r2_score(y_filtered, y_pred_filtered)
print(f"R2 (Filtered): {r2_filtered:.3f}")

In [ ]:
gold_df2 = gold_df.copy()
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(gold_df2[['word_len']], gold_df2['num_morphemes'])

gold_df2['predicted_rf'] = rf.predict(gold_df2[['word_len']])
gold_df2['residual_rf'] = gold_df2['num_morphemes'] - gold_df2['predicted_rf']

mse_full = mean_squared_error(gold_df2['num_morphemes'], gold_df2['predicted_rf'])
mae_full = mean_absolute_error(gold_df2['num_morphemes'], gold_df2['predicted_rf'])
r2_full = r2_score(gold_df2['num_morphemes'], gold_df2['predicted_rf'])

std_residual = gold_df2['residual_rf'].std()
filtered_df_rf = gold_df2[np.abs(gold_df2['residual_rf']) <= std_residual].copy()

rf_filtered = RandomForestRegressor(n_estimators=100, random_state=42)
X_filtered = filtered_df_rf[['word_len']]
y_filtered = filtered_df_rf['num_morphemes']
rf_filtered.fit(X_filtered, y_filtered)

filtered_df_rf['predicted_rf'] = rf_filtered.predict(X_filtered)
r2_filtered = r2_score(y_filtered, filtered_df_rf['predicted_rf'])
mse_filtered = mean_squared_error(y_filtered, filtered_df_rf['predicted_rf'])
mae_filtered = mean_absolute_error(y_filtered, filtered_df_rf['predicted_rf'])

print("Random Forest Evaluation (Before Outlier Removal):")
print(f"MSE: {mse_full:.3f}")
print(f"MAE: {mae_full:.3f}")
print(f"R²:  {r2_full:.3f}")

print("Random Forest Evaluation (After Outlier Removal):")
print(f"MSE: {mse_filtered:.3f}")
print(f"MAE: {mae_filtered:.3f}")
print(f"R²:  {r2_filtered:.3f}")

plt.figure(figsize=(10, 6))
sns.scatterplot(x=filtered_df_rf['word_len'], y=filtered_df_rf['num_morphemes'], alpha=0.5, label='Filtered Data')
sns.lineplot(x=filtered_df_rf['word_len'], y=filtered_df_rf['predicted_rf'], color='red', label='RF Prediction (Filtered)')
plt.xlabel('Word Length (Characters)')
plt.ylabel('Number of Morphemes')
plt.title('Random Forest Regression (Filtered)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(gold_df2[['word_len']])
model_poly = LinearRegression().fit(X_poly, gold_df2['num_morphemes'])
preds_poly = model_poly.predict(X_poly)
r2_before = r2_score(gold_df2['num_morphemes'], preds_poly)
print(f"Polynomial Regression R2 (Before Filtering): {r2_before:.3f}")

residuals_poly = gold_df2['num_morphemes'] - preds_poly
std_resid_poly = residuals_poly.std()
mask = np.abs(residuals_poly) <= std_resid_poly
filtered_df_poly = gold_df2[mask].copy()

X_filtered_poly = poly.fit_transform(filtered_df_poly[['word_len']])
model_poly_filtered = LinearRegression().fit(X_filtered_poly, filtered_df_poly['num_morphemes'])
preds_filtered = model_poly_filtered.predict(X_filtered_poly)
r2_after = r2_score(filtered_df_poly['num_morphemes'], preds_filtered)
print(f"Polynomial Regression R2 (After Filtering):  {r2_after:.3f}")

In [ ]:
linear_outliers = gold_df1[np.abs(gold_df1['residual']) > std_residual]
rf_outliers = gold_df2[np.abs(gold_df2['residual_rf']) > std_residual]
poly_outliers = gold_df2[np.abs(residuals_poly) > std_resid_poly]

all_outliers = pd.concat([linear_outliers, rf_outliers, poly_outliers])
all_outliers = all_outliers[['word_len', 'num_morphemes']].drop_duplicates()

In [ ]:
heatmap_data = gold_df.groupby(['word_len', 'num_morphemes']).size().unstack(fill_value=0)

outlier_coords = all_outliers[['word_len', 'num_morphemes']]
outlier_coords = outlier_coords[
    (outlier_coords['word_len'].isin(heatmap_data.index)) & 
    (outlier_coords['num_morphemes'].isin(heatmap_data.columns))
]
outlier_coords = outlier_coords.copy()
outlier_coords['freq'] = outlier_coords.apply(
    lambda row: heatmap_data.at[row['word_len'], row['num_morphemes']], axis=1
)

norm = plt.Normalize(vmin=heatmap_data.values.min(), vmax=heatmap_data.values.max())
colors = cm.Purples_r(norm(outlier_coords['freq'].values))

plt.figure(figsize=(10, 6))
ax = sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='Reds', cbar_kws={'label': 'Frequency'})
plt.title('Heatmap of Word Length vs. Morpheme Count with Outliers')
plt.xlabel('Number of Morphemes')
plt.ylabel('Word Length (Characters)')

for j, (_, row) in enumerate(outlier_coords.iterrows()):
    plt.scatter(
        x=row['num_morphemes'] + 0.5,
        y=row['word_len'] + 0.5,
        color=colors[j],
        s=100,
        marker='X',
        linewidths=2,
        label='Outlier' if j == 0 else ""
    )

plt.tight_layout()
plt.show()

In [ ]:
# =========================
# PREPARE SEGMENTATION FORMAT
# =========================
# Convert morpheme splits to segmentation format with '+' separators
# This format is used for the Transformer model (sequence-to-sequence)
# Example: "pi kuna s" -> "pi+kuna+s"
gold_df['segmentation'] = gold_df['Morph_split_str'].str.replace(' ', '+')

In [ ]:
gold_df.head()

In [ ]:
# =========================
# BUILD VOCABULARIES
# =========================
# Create character-level vocabularies for input (words) and output (segmented words)
# Input vocab: characters in words
# Output vocab: characters in segmented words (includes '+' separator)
special_tokens = ['<pad>', '<s>', '</s>']  # Padding, start, and end tokens
input_chars = sorted({ch for word in gold_df['Word'] for ch in word})
output_chars = sorted({ch for seg in gold_df['segmentation'] for ch in seg})
input_vocab = special_tokens + input_chars
output_vocab = special_tokens + output_chars
input2idx = {ch: idx for idx, ch in enumerate(input_vocab)}
output2idx = {ch: idx for idx, ch in enumerate(output_vocab)}
PAD_IN, START_IN, END_IN = input2idx['<pad>'], input2idx['<s>'], input2idx['</s>']
PAD_OUT, START_OUT, END_OUT = output2idx['<pad>'], output2idx['<s>'], output2idx['</s>']

# =========================
# DATASET CLASS
# =========================
# PyTorch Dataset for Transformer sequence-to-sequence training
# Each sample contains a word and its segmented version
class QuechuaSegDataset(Dataset):
    """
    Dataset for Quechua morphological segmentation.
    
    Args:
        df: DataFrame with 'Word' and 'segmentation' columns
        input2idx: Dictionary mapping input characters to indices
        output2idx: Dictionary mapping output characters to indices
        max_input_len: Maximum input sequence length (auto-computed if None)
        max_output_len: Maximum output sequence length (auto-computed if None)
    """
    def __init__(self, df, input2idx, output2idx, max_input_len=None, max_output_len=None):
        self.words = df['Word'].tolist()
        self.segs = df['segmentation'].tolist()
        self.input2idx = input2idx
        self.output2idx = output2idx
        # Add 2 for <s> and </s> tokens
        self.max_input_len = max_input_len or max(len(w) for w in self.words) + 2
        self.max_output_len = max_output_len or max(len(s) for s in self.segs) + 2

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words[idx]
        seg = self.segs[idx]
        # Encode source: <s> + chars + </s>, pad to max_input_len
        src = [self.input2idx.get(ch, PAD_IN) for ch in word]
        src = [START_IN] + src + [END_IN]
        src += [PAD_IN] * (self.max_input_len - len(src))
        # Encode target: <s> + segmented chars + </s>, pad to max_output_len
        tgt = [START_OUT] + [self.output2idx[ch] for ch in seg] + [END_OUT]
        tgt += [PAD_OUT] * (self.max_output_len - len(tgt))
        return torch.tensor(src), torch.tensor(tgt)

# =========================
# POSITIONAL ENCODING
# =========================
# Adds positional information to embeddings (required for Transformers)
class PositionalEncoding(nn.Module):
    """
    Sinusoidal positional encoding for Transformer models.
    Adds position-dependent information to token embeddings.
    """
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)  # Even dimensions: sin
        pe[:, 1::2] = torch.cos(pos * div)  # Odd dimensions: cos
        pe = pe.unsqueeze(1)  # Shape: (max_len, 1, d_model)
        self.register_buffer('pe', pe)
    def forward(self, x):
        # x shape: (seq_len, batch, d_model)
        return x + self.pe[:x.size(0)]

# =========================
# TRANSFORMER MODEL ARCHITECTURE
# =========================
# Encoder-decoder Transformer for sequence-to-sequence morphological segmentation
class MorphSegModel(nn.Module):
    """
    Transformer encoder-decoder model for morphological segmentation.
    
    Architecture:
    1. Encoder: Processes input word character-by-character
    2. Decoder: Generates segmented word character-by-character (with '+' separators)
    3. Output projection: Maps decoder hidden states to output vocabulary
    
    Args:
        in_vocab: Size of input vocabulary (characters in words)
        out_vocab: Size of output vocabulary (characters in segmented words, including '+')
        d_model: Model dimension (embedding size)
        ff: Feed-forward network dimension
        heads: Number of attention heads
        layers: Number of encoder/decoder layers
        drop: Dropout rate
    """
    def __init__(self, in_vocab, out_vocab, d_model=64, ff=128, heads=2, layers=1, drop=0.0):
        super().__init__()
        # Encoder: processes input word
        self.enc_embed = nn.Embedding(in_vocab, d_model, padding_idx=PAD_IN)
        self.pos_enc = PositionalEncoding(d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model, heads, ff, drop, batch_first=False)
        self.encoder = nn.TransformerEncoder(enc_layer, layers)

        # Decoder: generates segmented word
        self.dec_embed = nn.Embedding(out_vocab, d_model, padding_idx=PAD_OUT)
        self.pos_dec = PositionalEncoding(d_model)
        dec_layer = nn.TransformerDecoderLayer(d_model, heads, ff, drop, batch_first=False)
        self.decoder = nn.TransformerDecoder(dec_layer, layers)

        # Output projection: maps to output vocabulary
        self.out_proj = nn.Linear(d_model, out_vocab)

    def forward(self, src, tgt, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
        """
        Forward pass through the Transformer.
        
        Args:
            src: Input word tokens (seq_len, batch)
            tgt: Target segmented word tokens (seq_len, batch)
            tgt_mask: Causal mask for decoder (prevents looking ahead)
            src_key_padding_mask: Mask for padding in source
            tgt_key_padding_mask: Mask for padding in target
        
        Returns:
            Logits over output vocabulary (seq_len, batch, out_vocab)
        """
        # Encode source word
        e_src = self.pos_enc(self.enc_embed(src))
        memory = self.encoder(e_src, src_key_padding_mask=src_key_padding_mask)
        
        # Decode segmented word
        e_tgt = self.pos_dec(self.dec_embed(tgt))
        out = self.decoder(e_tgt, memory, tgt_mask=tgt_mask,
                           memory_key_padding_mask=src_key_padding_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask)
        
        # Project to output vocabulary
        return self.out_proj(out)


In [ ]:
# =========================
# DATASET SPLIT AND DATALOADERS
# =========================
# Split data into 80% training and 20% validation
# Create DataLoaders for batching
dataset = QuechuaSegDataset(gold_df, input2idx, output2idx)
n_train = int(0.8 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [n_train, len(dataset) - n_train])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
print(f"Training samples: {len(train_dataset):,}")
print(f"Validation samples: {len(val_dataset):,}")

In [ ]:
# =========================
# MODEL CHECKPOINTING FUNCTIONS
# =========================
# Functions to save and load trained models to avoid retraining

def generate_model_id(d_model, ff, heads, layers, drop, epochs, batch_size, lr, in_vocab_size, out_vocab_size):
    """
    Generate a unique identifier for a model based on its training parameters.
    
    Args:
        All training hyperparameters and vocabulary sizes
    
    Returns:
        A string identifier (hash) for the model
    """
    params_dict = {
        'd_model': d_model,
        'ff': ff,
        'heads': heads,
        'layers': layers,
        'drop': drop,
        'epochs': epochs,
        'batch_size': batch_size,
        'lr': lr,
        'in_vocab_size': in_vocab_size,
        'out_vocab_size': out_vocab_size
    }
    params_str = json.dumps(params_dict, sort_keys=True)
    model_id = hashlib.md5(params_str.encode()).hexdigest()[:16]
    return model_id

def save_model_checkpoint(model, input2idx, output2idx, model_id, models_folder=MODELS_FOLDER):
    """
    Save model checkpoint to the models folder.
    
    Args:
        model: Trained MorphSegModel
        input2idx: Input vocabulary mapping
        output2idx: Output vocabulary mapping
        model_id: Unique identifier for this model
        models_folder: Folder to save models in
    """
    model_dir = os.path.join(models_folder, model_id)
    os.makedirs(model_dir, exist_ok=True)
    
    checkpoint_path = os.path.join(model_dir, "transformer_morph_seg.pt")
    torch.save({
        "model_state": model.state_dict(),
        "input2idx": input2idx,
        "output2idx": output2idx
    }, checkpoint_path)
    
    # Save metadata
    metadata_path = os.path.join(model_dir, "metadata.json")
    with open(metadata_path, "w") as f:
        json.dump({
            'model_id': model_id,
            'in_vocab_size': len(input2idx),
            'out_vocab_size': len(output2idx),
            'model_name': MODEL_NAME
        }, f, indent=2)
    
    print(f"Model checkpoint saved to {model_dir}")
    return model_dir

def load_model_checkpoint(model_id, models_folder=MODELS_FOLDER):
    """
    Load model checkpoint from the models folder.
    
    Args:
        model_id: Unique identifier for the model
        models_folder: Folder where models are saved
    
    Returns:
        Dictionary with 'model_state', 'input2idx', 'output2idx', 'checkpoint_path', 'model_dir' or None if not found
    """
    model_dir = os.path.join(models_folder, model_id)
    checkpoint_path = os.path.join(model_dir, "transformer_morph_seg.pt")
    
    if not os.path.exists(checkpoint_path):
        return None
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    print(f"Model checkpoint loaded from {model_dir}")
    return {
        'model_state': checkpoint['model_state'],
        'input2idx': checkpoint['input2idx'],
        'output2idx': checkpoint['output2idx'],
        'checkpoint_path': checkpoint_path,
        'model_dir': model_dir
    }

# =========================
# MODEL INITIALIZATION
# =========================
# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model hyperparameters
D_MODEL = 64
FF = 128
HEADS = 2
LAYERS = 1
DROP = 0.0
EPOCHS = 15
BATCH_SIZE = 16
LR = 1e-4

# Generate model identifier
model_id = generate_model_id(D_MODEL, FF, HEADS, LAYERS, DROP, EPOCHS, BATCH_SIZE, LR, 
                              len(input_vocab), len(output_vocab))

# Try to load existing model
print(f"Checking for existing model with ID: {model_id}")
loaded = load_model_checkpoint(model_id, models_folder=MODELS_FOLDER)

if loaded is not None:
    print(f"Found existing model! Loading from {loaded['model_dir']}")
    input2idx = loaded['input2idx']
    output2idx = loaded['output2idx']
    # Recompute special token indices from loaded vocabularies
    PAD_IN, START_IN, END_IN = input2idx['<pad>'], input2idx['<s>'], input2idx['</s>']
    PAD_OUT, START_OUT, END_OUT = output2idx['<pad>'], output2idx['<s>'], output2idx['</s>']
    model = MorphSegModel(len(input2idx), len(output2idx), d_model=D_MODEL, ff=FF, 
                          heads=HEADS, layers=LAYERS, drop=DROP).to(device)
    model.load_state_dict(loaded['model_state'])
    model.eval()
    print("Model loaded successfully. Skipping training.")
else:
    print(f"No existing model found. Training new model...")
    model = MorphSegModel(len(input_vocab), len(output_vocab), d_model=D_MODEL, ff=FF, 
                          heads=HEADS, layers=LAYERS, drop=DROP).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_OUT)
    optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
# =========================
# TRAINING LOOP
# =========================
# Train the Transformer model for specified number of epochs
# Only runs if no existing model was found

if loaded is None:
    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss = 0
        for src_batch, tgt_batch in train_loader:
            # 1) Move batch to device
            src_batch = src_batch.to(device)           # (batch, src_len)
            tgt_batch = tgt_batch.to(device)           # (batch, tgt_len)

            # 2) Build padding masks (True for padding tokens)
            src_pad_mask = (src_batch == PAD_IN)       # (batch, src_len)
            tgt_pad_mask = (tgt_batch == PAD_OUT)     # (batch, tgt_len)

            # 3) Transpose to (seq_len, batch) as expected by nn.Transformer
            src = src_batch.transpose(0, 1)            # (src_len, batch)
            tgt = tgt_batch.transpose(0, 1)            # (tgt_len, batch)
            tgt_input = tgt[:-1, :]                    # Input to decoder (shifted)
            tgt_output = tgt[1:, :]                    # Target output (shifted)

            # 4) Causal mask for decoder (prevents looking at future tokens)
            tgt_mask = nn.Transformer.generate_square_subsequent_mask(
                           tgt_input.size(0)
                       ).to(device)

            # 5) Forward pass: predict next token in segmented sequence
            logits = model(
                src,
                tgt_input,
                tgt_mask=tgt_mask,
                src_key_padding_mask=src_pad_mask,
                tgt_key_padding_mask=tgt_pad_mask[:, :-1]
            )
            
            # 6) Compute loss (cross-entropy over output vocabulary)
            loss = criterion(
                logits.reshape(-1, logits.size(-1)),  # Flatten to (seq*batch, vocab)
                tgt_output.reshape(-1)                # Flatten to (seq*batch,)
            )

            # 7) Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch:02d} — Train Loss: {avg_loss:.4f}")
    
    # Save model after training
    save_model_checkpoint(model, input2idx, output2idx, model_id, models_folder=MODELS_FOLDER)
    print(f"\nTraining complete! Model saved with ID: {model_id}")
else:
    print("Using loaded model. Training skipped.")

In [ ]:
# ===================================================================
# K-FOLD CROSS-VALIDATION FUNCTION
# ===================================================================
# This function performs k-fold cross-validation on the training data
# It splits the data into k folds and trains/evaluates on each fold
# For each fold, it trains a Transformer encoder-decoder model

from sklearn.model_selection import KFold

def run_kfold_cross_validation(
    df,
    n_folds=5,
    d_model=64,
    ff=128,
    heads=2,
    layers=1,
    drop=0.0,
    epochs=15,
    batch_size=16,
    lr=1e-4,
    random_state=42,
    device=device
):
    """
    Perform k-fold cross-validation on the training data with Transformer model.
    
    Args:
        df: Training DataFrame with 'Word' and 'segmentation' columns
        n_folds: Number of folds for cross-validation (default: 5)
        d_model: Model dimension (embedding size)
        ff: Feed-forward network dimension
        heads: Number of attention heads
        layers: Number of encoder/decoder layers
        drop: Dropout rate
        epochs: Number of training epochs per fold
        batch_size: Training batch size
        lr: Learning rate
        random_state: Random seed for reproducibility
        device: Device to run training on
    
    Returns:
        Dictionary containing:
        - fold_results: List of results for each fold
        - mean_metrics: Average metrics across all folds
        - std_metrics: Standard deviation of metrics across folds
        - best_fold_idx: Index of the fold with best validation loss
    """
    print(f"\n{'='*80}")
    print(f"K-FOLD CROSS-VALIDATION (k={n_folds}) WITH TRANSFORMER")
    print(f"{'='*80}")
    
    # Create k-fold splitter
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    indices = np.arange(len(df))
    
    fold_results = []
    all_metrics = {
        'val_loss': [],
        'exact_match': []
    }
    
    # Train and evaluate on each fold
    for fold_idx, (train_indices, val_indices) in enumerate(kfold.split(indices), 1):
        print(f"\n{'─'*80}")
        print(f"FOLD {fold_idx}/{n_folds}")
        print(f"{'─'*80}")
        print(f"Train samples: {len(train_indices)}, Validation samples: {len(val_indices)}")
        
        # Split data into train and validation
        train_df_fold = df.iloc[train_indices].reset_index(drop=True)
        val_df_fold = df.iloc[val_indices].reset_index(drop=True)
        
        # Build vocabularies from training fold only
        print(f"  Building vocabularies from training fold...")
        special_tokens = ['<pad>', '<s>', '</s>']
        input_chars_fold = sorted({ch for word in train_df_fold['Word'] for ch in word})
        output_chars_fold = sorted({ch for seg in train_df_fold['segmentation'] for ch in seg})
        input_vocab_fold = special_tokens + input_chars_fold
        output_vocab_fold = special_tokens + output_chars_fold
        input2idx_fold = {ch: idx for idx, ch in enumerate(input_vocab_fold)}
        output2idx_fold = {ch: idx for idx, ch in enumerate(output_vocab_fold)}
        PAD_IN_FOLD = input2idx_fold['<pad>']
        START_IN_FOLD = input2idx_fold['<s>']
        END_IN_FOLD = input2idx_fold['</s>']
        PAD_OUT_FOLD = output2idx_fold['<pad>']
        START_OUT_FOLD = output2idx_fold['<s>']
        END_OUT_FOLD = output2idx_fold['</s>']
        
        # Create datasets
        train_dataset_fold = QuechuaSegDataset(train_df_fold, input2idx_fold, output2idx_fold)
        val_dataset_fold = QuechuaSegDataset(val_df_fold, input2idx_fold, output2idx_fold)
        train_loader_fold = DataLoader(train_dataset_fold, batch_size=batch_size, shuffle=True)
        val_loader_fold = DataLoader(val_dataset_fold, batch_size=batch_size, shuffle=False)
        
        # Create model
        model_fold = MorphSegModel(
            len(input_vocab_fold), 
            len(output_vocab_fold), 
            d_model=d_model, 
            ff=ff, 
            heads=heads, 
            layers=layers, 
            drop=drop
        ).to(device)
        
        # Create loss function and optimizer
        criterion_fold = nn.CrossEntropyLoss(ignore_index=PAD_OUT_FOLD)
        optimizer_fold = optim.Adam(model_fold.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)
        
        # Training loop
        best_val_loss = float('inf')
        best_epoch = 0
        best_exact_match = 0.0
        
        for epoch in range(1, epochs+1):
            # Training phase
            model_fold.train()
            total_loss = 0.0
            for src_batch, tgt_batch in train_loader_fold:
                src_batch = src_batch.to(device)
                tgt_batch = tgt_batch.to(device)
                
                src_pad_mask = (src_batch == PAD_IN_FOLD)
                tgt_pad_mask = (tgt_batch == PAD_OUT_FOLD)
                
                src = src_batch.transpose(0, 1)
                tgt = tgt_batch.transpose(0, 1)
                tgt_input = tgt[:-1, :]
                tgt_output = tgt[1:, :]
                
                tgt_mask = nn.Transformer.generate_square_subsequent_mask(
                    tgt_input.size(0)
                ).to(device)
                
                logits = model_fold(
                    src,
                    tgt_input,
                    tgt_mask=tgt_mask,
                    src_key_padding_mask=src_pad_mask,
                    tgt_key_padding_mask=tgt_pad_mask[:, :-1]
                )
                
                loss = criterion_fold(
                    logits.reshape(-1, logits.size(-1)),
                    tgt_output.reshape(-1)
                )
                
                optimizer_fold.zero_grad()
                loss.backward()
                optimizer_fold.step()
                
                total_loss += loss.item()
            
            train_loss = total_loss / len(train_loader_fold)
            
            # Validation phase
            model_fold.eval()
            val_loss = 0.0
            exact_matches = 0
            total_val = 0
            
            # Build reverse vocabulary for inference
            idx2output_fold = {idx: ch for ch, idx in output2idx_fold.items()}
            
            # Define fold-specific segment_word function
            def segment_word_fold(word, model, in2idx, out2idx, idx2out, max_len=50):
                """Fold-specific version of segment_word using fold-specific constants."""
                model.eval()
                src_idx = [START_IN_FOLD] + [in2idx.get(ch, PAD_IN_FOLD) for ch in word] + [END_IN_FOLD]
                src = torch.tensor(src_idx).unsqueeze(1).to(device)
                src_pad = (src.squeeze(1) == PAD_IN_FOLD).unsqueeze(0)
                
                with torch.no_grad():
                    mem = model.pos_enc(model.enc_embed(src))
                    mem = model.encoder(mem, src_key_padding_mask=src_pad)
                
                out_ids = [START_OUT_FOLD]
                for step in range(max_len):
                    tgt = torch.tensor(out_ids).unsqueeze(1).to(device)
                    mask = nn.Transformer.generate_square_subsequent_mask(len(out_ids)).to(device)
                    with torch.no_grad():
                        dec_out = model.pos_dec(model.dec_embed(tgt))
                        dec = model.decoder(dec_out, mem, tgt_mask=mask, memory_key_padding_mask=src_pad)
                        logits = model.out_proj(dec)
                    
                    probs = torch.softmax(logits[-1, 0], dim=0)
                    nxt = logits[-1, 0].argmax().item()
                    
                    current_output = ''.join(idx2out[i] for i in out_ids[1:])
                    current_chars = len(current_output.replace('+', ''))
                    input_chars = len(word)
                    min_expected_chars = input_chars
                    
                    if nxt == END_OUT_FOLD:
                        end_prob = probs[END_OUT_FOLD].item()
                        if current_chars >= min_expected_chars or end_prob > 0.8:
                            break
                        else:
                            top_probs, top_indices = torch.topk(probs, k=3)
                            for idx in top_indices:
                                if idx.item() != END_OUT_FOLD:
                                    nxt = idx.item()
                                    break
                    
                    out_ids.append(nxt)
                
                result = ''.join(idx2out[i] for i in out_ids[1:])
                return result
            
            with torch.no_grad():
                # Compute validation loss
                for src_batch, tgt_batch in val_loader_fold:
                    src_batch = src_batch.to(device)
                    tgt_batch = tgt_batch.to(device)
                    
                    src_pad_mask = (src_batch == PAD_IN_FOLD)
                    tgt_pad_mask = (tgt_batch == PAD_OUT_FOLD)
                    
                    src = src_batch.transpose(0, 1)
                    tgt = tgt_batch.transpose(0, 1)
                    tgt_input = tgt[:-1, :]
                    tgt_output = tgt[1:, :]
                    
                    tgt_mask = nn.Transformer.generate_square_subsequent_mask(
                        tgt_input.size(0)
                    ).to(device)
                    
                    logits = model_fold(
                        src,
                        tgt_input,
                        tgt_mask=tgt_mask,
                        src_key_padding_mask=src_pad_mask,
                        tgt_key_padding_mask=tgt_pad_mask[:, :-1]
                    )
                    
                    loss = criterion_fold(
                        logits.reshape(-1, logits.size(-1)),
                        tgt_output.reshape(-1)
                    )
                    val_loss += loss.item()
                
                # Compute exact match accuracy on validation set
                for i in range(len(val_dataset_fold)):
                    word = val_df_fold.iloc[i]['Word']
                    gold_seg = val_df_fold.iloc[i]['segmentation']
                    
                    # Predict segmentation using fold-specific function
                    seg_str = segment_word_fold(
                        word, model_fold, input2idx_fold, output2idx_fold, 
                        idx2output_fold, max_len=50
                    )
                    
                    if seg_str == gold_seg:
                        exact_matches += 1
                    total_val += 1
            
            val_loss = val_loss / len(val_loader_fold)
            exact_match_rate = exact_matches / total_val if total_val > 0 else 0.0
            
            print(f"  Epoch {epoch:02d} | train_loss={train_loss:.4f}  val_loss={val_loss:.4f}  exact_match={exact_match_rate:.3f}")
            
            # Track best validation performance
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_exact_match = exact_match_rate
                best_epoch = epoch
        
        print(f"\n  Best epoch: {best_epoch}")
        print(f"  Best validation: Loss={best_val_loss:.4f}  Exact Match={best_exact_match:.3f}")
        
        # Store fold results
        fold_result = {
            'fold': fold_idx,
            'val_loss': best_val_loss,
            'exact_match': best_exact_match,
            'best_epoch': best_epoch
        }
        fold_results.append(fold_result)
        
        # Collect metrics for averaging
        all_metrics['val_loss'].append(best_val_loss)
        all_metrics['exact_match'].append(best_exact_match)
    
    # Calculate mean and std across folds
    mean_metrics = {
        'val_loss': np.mean(all_metrics['val_loss']),
        'exact_match': np.mean(all_metrics['exact_match'])
    }
    
    std_metrics = {
        'val_loss': np.std(all_metrics['val_loss']),
        'exact_match': np.std(all_metrics['exact_match'])
    }
    
    # Find best fold (lowest validation loss)
    best_fold_idx = min(range(len(fold_results)), key=lambda i: fold_results[i]['val_loss'])
    
    # Print summary
    print(f"\n{'='*80}")
    print(f"K-FOLD CROSS-VALIDATION SUMMARY")
    print(f"{'='*80}")
    print(f"\nPer-fold results:")
    for result in fold_results:
        print(f"  Fold {result['fold']}: "
              f"Loss={result['val_loss']:.4f}, "
              f"Exact Match={result['exact_match']:.3f}")
    
    print(f"\nMean ± Std across {n_folds} folds:")
    print(f"  Validation Loss:   {mean_metrics['val_loss']:.4f} ± {std_metrics['val_loss']:.4f}")
    print(f"  Exact Match Rate:  {mean_metrics['exact_match']:.3f} ± {std_metrics['exact_match']:.3f}")
    print(f"\nBest fold: Fold {fold_results[best_fold_idx]['fold']} "
          f"(Loss: {fold_results[best_fold_idx]['val_loss']:.4f}, "
          f"Exact Match: {fold_results[best_fold_idx]['exact_match']:.3f})")
    print(f"{'='*80}\n")
    
    return {
        'fold_results': fold_results,
        'mean_metrics': mean_metrics,
        'std_metrics': std_metrics,
        'best_fold_idx': best_fold_idx,
        'all_metrics': all_metrics
    }


In [ ]:
# ===================================================================
# K-FOLD CROSS-VALIDATION DEMONSTRATION
# ===================================================================
# This cell demonstrates how to use k-fold cross-validation to evaluate
# model performance more robustly by training on multiple train/val splits
# Each fold trains its own Transformer encoder-decoder model

# Run 5-fold cross-validation on the training data
kfold_results = run_kfold_cross_validation(
    df=gold_df,  # Use the full gold_df for cross-validation
    n_folds=5,  # Number of folds
    d_model=D_MODEL,
    ff=FF,
    heads=HEADS,
    layers=LAYERS,
    drop=DROP,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    random_state=42,  # Use fixed seed for reproducibility
    device=device
)

# The results dictionary contains:
# - fold_results: List of results for each fold
# - mean_metrics: Average metrics across all folds
# - std_metrics: Standard deviation of metrics across folds
# - best_fold_idx: Index of the fold with best validation loss
# - all_metrics: Raw metrics from all folds

print("\nK-fold cross-validation completed!")
print(f"Average exact match rate: {kfold_results['mean_metrics']['exact_match']:.3f} ± {kfold_results['std_metrics']['exact_match']:.3f}")
print(f"Average validation loss: {kfold_results['mean_metrics']['val_loss']:.4f} ± {kfold_results['std_metrics']['val_loss']:.4f}")


In [ ]:
# =========================
# INFERENCE FUNCTION
# =========================
# Function to segment a word using the trained Transformer model
def segment_word(word, model, in2idx, out2idx, idx2out, max_len=50, debug=False):
    """
    Segment a word using the trained Transformer model.
    
    Args:
        word: Input word string to segment
        model: Trained MorphSegModel
        in2idx: Input vocabulary mapping
        out2idx: Output vocabulary mapping
        idx2out: Reverse output vocabulary mapping (index to character)
        max_len: Maximum output sequence length
        debug: If True, print debugging information about token generation
    
    Returns:
        Segmented word string (e.g., "pi+kuna+s")
    """
    model.eval()
    # Encode input word: <s> + chars + </s>
    src_idx = [START_IN] + [in2idx.get(ch, PAD_IN) for ch in word] + [END_IN]
    src = torch.tensor(src_idx).unsqueeze(1).to(device)  # (seq_len, 1)
    src_pad = (src.squeeze(1) == PAD_IN).unsqueeze(0)    # (1, seq_len)
    
    # Encode source word
    with torch.no_grad():
        mem = model.pos_enc(model.enc_embed(src))
        mem = model.encoder(mem, src_key_padding_mask=src_pad)
    
    # Decode segmented word autoregressively
    out_ids = [START_OUT]
    if debug:
        print(f"Segmenting '{word}':")
        print(f"  Generated tokens so far: ", end="")
    
    for step in range(max_len):
        tgt = torch.tensor(out_ids).unsqueeze(1).to(device)  # (seq_len, 1)
        mask = nn.Transformer.generate_square_subsequent_mask(len(out_ids)).to(device)
        with torch.no_grad():
            dec_out = model.pos_dec(model.dec_embed(tgt))
            dec = model.decoder(dec_out, mem, tgt_mask=mask, memory_key_padding_mask=src_pad)
            logits = model.out_proj(dec)
        
        # Get probabilities for top predictions
        probs = torch.softmax(logits[-1, 0], dim=0)
        top_probs, top_indices = torch.topk(probs, k=3)
        
        # Predict next token (greedy decoding)
        nxt = logits[-1, 0].argmax().item()
        
        if debug:
            top_chars = [idx2out[idx.item()] for idx in top_indices]
            print(f"\n  Step {step+1}: top predictions = {list(zip(top_chars, top_probs.tolist()))}")
            print(f"    -> Selected: '{idx2out[nxt]}' (prob={probs[nxt].item():.4f})")
        
        # Smart stopping criterion: don't stop too early
        # Calculate current output length (excluding '+' separators)
        current_output = ''.join(idx2out[i] for i in out_ids[1:])
        current_chars = len(current_output.replace('+', ''))
        input_chars = len(word)
        
        # Don't stop if:
        # 1. We haven't generated at least as many characters as the input (accounting for separators)
        # 2. The probability of END_OUT is not very high (below 0.8) AND we're still short
        min_expected_chars = input_chars  # Should generate at least as many chars as input
        
        if nxt == END_OUT:
            # Only stop if we've generated enough characters OR END_OUT probability is very high
            end_prob = probs[END_OUT].item()
            if current_chars >= min_expected_chars or end_prob > 0.8:
                if debug:
                    print(f"  Stopped at END_OUT token (chars: {current_chars}/{min_expected_chars}, prob: {end_prob:.4f})")
                break
            else:
                # Force continue: select the second-best token if it's not END_OUT
                if debug:
                    print(f"  END_OUT predicted but too early (chars: {current_chars}/{min_expected_chars}), forcing continuation...")
                # Find the best non-END_OUT token
                for idx in top_indices:
                    if idx.item() != END_OUT:
                        nxt = idx.item()
                        if debug:
                            print(f"    -> Forced selection: '{idx2out[nxt]}' (prob={probs[nxt].item():.4f})")
                        break
        
        out_ids.append(nxt)
        
        if debug:
            current_seg = ''.join(idx2out[i] for i in out_ids[1:])
            print(f"    Current segmentation: '{current_seg}'")
    
    # Convert indices to characters (skip START_OUT token)
    result = ''.join(idx2out[i] for i in out_ids[1:])
    
    if debug:
        print(f"  Final result: '{result}'")
        print(f"  Expected length check: input '{word}' ({len(word)} chars) -> output '{result}' ({len(result.replace('+', ''))} chars)")
    
    return result

In [ ]:
# =========================
# BUILD REVERSE VOCABULARY MAPPING
# =========================
# Create index-to-character mapping for output (needed for inference)
idx2output = {idx: ch for ch, idx in output2idx.items()}

# Example usage with debugging to see what's happening:
print("Example segmentation with debugging:")
test_word = "pikunas"
segmented = segment_word(test_word, model, input2idx, output2idx, idx2output, debug=True)
print(f"\nFinal result: {test_word} -> {segmented}")

# Check what the gold standard segmentation should be
if 'gold_df' in locals():
    gold_seg = gold_df[gold_df['Word'] == test_word]
    if len(gold_seg) > 0:
        print(f"\nGold standard segmentation(s):")
        for idx, row in gold_seg.iterrows():
            print(f"  {row['Morph_split']} -> {row['segmentation']}")

In [ ]:
# =========================
# EVALUATION ON TEST SET
# =========================
# Load test data and evaluate the model with comprehensive metrics
# Similar to segmenter.ipynb evaluation

print("Loading test data...")
test_df = pd.read_parquet(os.path.join(DATA_FOLDER, "cleaned_data_df.parquet"))
print(f"Loaded {len(test_df):,} test examples")

# Build reverse vocabulary mapping
idx2output = {idx: ch for ch, idx in output2idx.items()}

# =========================
# EVALUATION HELPER FUNCTIONS
# =========================
# Functions for evaluating segmentation accuracy

def parse_segmentation(seg_str):
    """
    Parse segmented string (e.g., "pi+kuna+s") into list of morphemes.
    
    Args:
        seg_str: Segmented string with '+' separators
    
    Returns:
        List of morpheme strings
    """
    if not seg_str:
        return []
    return seg_str.split('+')

def is_correct_prediction(predicted, gold_variants):
    """
    Check if predicted segmentation exactly matches any gold variant.
    
    Args:
        predicted: List of predicted morphemes
        gold_variants: List of gold segmentation variants (each is a list of morphemes)
    
    Returns:
        True if prediction matches any gold variant, False otherwise
    """
    # Normalize gold_variants (handle numpy arrays, nested structures)
    if gold_variants is None:
        return False
    
    if isinstance(gold_variants, np.ndarray):
        gold_variants = gold_variants.tolist()
    
    if isinstance(gold_variants, list):
        normalized = []
        for variant in gold_variants:
            if isinstance(variant, np.ndarray):
                normalized.append(variant.tolist())
            elif isinstance(variant, list):
                normalized.append([item.tolist() if isinstance(item, np.ndarray) else item for item in variant])
            else:
                normalized.append(variant)
        gold_variants = normalized
    
    return any(predicted == variant for variant in gold_variants)

def split_count_metrics(predicted_segments, gold_variants):
    """
    Compute split-count accuracy variants:
    - Exact: same number of morphemes as any gold variant
    - +1: one more split than any gold variant
    - -1: one fewer split than any gold variant
    - ±1: difference ≤ 1 with any gold variant
    """
    pred_count = len(predicted_segments)
    
    # Normalize gold_variants
    if gold_variants is None:
        return {"Exact": False, "+1": False, "-1": False, "±1": False}
    
    if isinstance(gold_variants, np.ndarray):
        gold_variants = gold_variants.tolist()
    
    if isinstance(gold_variants, list):
        normalized = []
        for variant in gold_variants:
            if isinstance(variant, np.ndarray):
                normalized.append(variant.tolist())
            elif isinstance(variant, list):
                normalized.append([item.tolist() if isinstance(item, np.ndarray) else item for item in variant])
            else:
                normalized.append(variant)
        gold_variants = normalized
    
    gold_counts = [len(gold) for gold in gold_variants]

    exact = any(pred_count == g for g in gold_counts)
    plus1 = any(pred_count == g + 1 for g in gold_counts)
    minus1 = any(pred_count == g - 1 for g in gold_counts)
    pm1 = any(abs(pred_count - g) <= 1 for g in gold_counts)

    return {"Exact": exact, "+1": plus1, "-1": minus1, "±1": pm1}

# =========================
# EVALUATION LOOP
# =========================
# Predict segmentations for all test words and compute metrics

records = []
all_words = test_df["Word"].tolist()

print("Predicting segmentations for test words...")
for word in all_words:
    # Predict segmentation
    seg_str = segment_word(word, model, input2idx, output2idx, idx2output)
    predicted_segments = parse_segmentation(seg_str)
    
    # Get gold variants
    gold_variants = test_df[test_df["Word"] == word]["Gold"].iloc[0] if len(test_df[test_df["Word"] == word]) > 0 else []
    
    # Exact match accuracy
    correct_exact = is_correct_prediction(predicted_segments, gold_variants)
    
    # Split-count metrics
    split_metrics = split_count_metrics(predicted_segments, gold_variants)
    
    records.append({
        "Word": word,
        "Prediction": predicted_segments,
        "Gold": gold_variants,
        "CorrectExactSeg": correct_exact,
        "CorrectSplitCount": split_metrics["Exact"],
        "SplitCount+1": split_metrics["+1"],
        "SplitCount-1": split_metrics["-1"],
        "SplitCount±1": split_metrics["±1"],
        "OverlapExactAndSplit": correct_exact and split_metrics["Exact"]
    })

results_df = pd.DataFrame(records)

# =========================
# COMPUTE AGGREGATE METRICS
# =========================
# Calculate overall accuracy and split-count metrics

accuracy = results_df["CorrectExactSeg"].mean()
split_exact_acc = results_df["CorrectSplitCount"].mean()
split_plus1_acc = results_df["SplitCount+1"].mean()
split_minus1_acc = results_df["SplitCount-1"].mean()
split_pm1_acc = results_df["SplitCount±1"].mean()
overlap_accuracy = results_df["OverlapExactAndSplit"].mean()

print(f"\n=== Evaluation Results ===")
print(f"Exact segmentation accuracy: {accuracy:.4f}")
print(f"\n=== Split-count metrics ===")
print(f"Split-count (Exact):          {split_exact_acc:.4f}")
print(f"Split-count (+1):             {split_plus1_acc:.4f}")
print(f"Split-count (−1):              {split_minus1_acc:.4f}")
print(f"Split-count (±1):              {split_pm1_acc:.4f}")
print(f"Overlap (Exact ∩ Split):      {overlap_accuracy:.4f}")

# =========================
# SAVE EVALUATION RESULTS
# =========================
# Save evaluation results to the data folder with a descriptive filename
results_output_path = os.path.join(DATA_FOLDER, "transformer_eval_results.csv")
results_df.to_csv(results_output_path, index=False)
print(f"\nEvaluation results saved to {results_output_path}")


In [ ]:
# =========================
# EXAMPLE USAGE
# =========================
# Test the model on a few example words
test_words = ["pikunas", "rikuchkani", "ñichkanchus"]
print("Example segmentations:")
for word in test_words:
    segmented = segment_word(word, model, input2idx, output2idx, idx2output)
    print(f"  {word} -> {segmented}")